In [1]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy.fftpack import dct
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, LSTM, Bidirectional

import seaborn as sns
import matplotlib.pyplot as plt

# PLP Module

In [2]:
def extract_plp_features(audio_path, sr=16000, order=13):

    # Pre-emphasis
    pre_emphasis = 0.97
    emphasized_audio = np.append(audio_path[0], audio_path[1:] - pre_emphasis * audio_path[:-1])

    # Framing
    frame_size = 0.025
    frame_stride = 0.01
    frame_length, frame_step = frame_size * sr, frame_stride * sr
    signal_length = len(emphasized_audio)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step)) + 1
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_audio, z)

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(
        np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    
    # Windowing
    # frames *= np.hamming(frame_length)
    frames *= np.hanning(frame_length)
    # frames *= np.blackman(frame_length)

    # Fourier-Transform and Power Spectrum
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))

    # Filter Banks
    nfilt = 26
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (sr / 2) / 700))
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
    hz_points = (700 * (10**(mel_points / 2595) - 1))
    bin = np.floor((NFFT + 1) * hz_points / sr)
    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])
        f_m = int(bin[m])
        f_m_plus = int(bin[m + 1])
        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
    filter_banks = 20 * np.log10(filter_banks)

    # Cepstral Coefficients
    num_ceps = order
    cep_lifter = 22
    cepstral_coefficients = dct(filter_banks, type=2, axis=1, norm='ortho')[:, :num_ceps]

    # Liftering
    nframes, ncoeff = cepstral_coefficients.shape
    n = np.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
    cepstral_coefficients *= lift

    return cepstral_coefficients

# Code

In [3]:
data_path = pd.read_csv("data_path.csv")

In [4]:
# I want to remove suprise and calm emotions from the dataset
data_path = data_path[(data_path.Emotions != 'calm') & (data_path.Emotions != 'surprise')]

In [ ]:
plt.title('Count of Emotions', size=16)
sns.countplot(data_path.Emotions)
plt.ylabel('Count', size=12)
plt.xlabel('Emotions', size=12)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.show()

In [86]:
# Padding function
def pad_features(features, max_len=120):
    return pad_sequences(features, maxlen=max_len, padding='post', dtype='float32')

In [87]:
X, Y = [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    data, sample_rate = librosa.load(path)
    # plp = librosa.feature.plp(y=data, sr=sample_rate, n_plp=13).T
    plp = extract_plp_features(data, sr=sample_rate)
    X.append(plp)       
    Y.append(emotion)
    
X = pad_features(X, max_len=100)
X = np.array(X)

In [88]:
Y = np.array(Y)

In [89]:
np.save('X_plp.npy', X)
np.save('Y_plp_labels.npy', Y)

In [90]:
# Load saved X and Y
X = np.load('X_plp.npy', allow_pickle=True)
Y = np.load('Y_plp_labels.npy', allow_pickle=True)

In [ ]:
Y.shape
X.shape

In [ ]:
# I want a list that has the unique emotions
unique_emotions = data_path.Emotions.unique()
# unique_emotions
# convert to list
unique_emotions = list(unique_emotions)
unique_emotions
# count the number of unique emotions
n_classes = len(unique_emotions)
n_classes, unique_emotions
emotions = np.array(unique_emotions)
unique_emotions

In [42]:
# As this is a multiclass classification problem onehotencoding our Y.
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [11]:
# from tensorflow.keras.regularizers import l2

# # Train the audio model
# def train_audio_model(X_train, y_train, X_test, y_test, num_classes=8):
    
#     # Create the model
#     input_shape = (120, X_train.shape[2])  
#     audio_model = Sequential([
#         Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
#         MaxPooling1D(pool_size=2),
#         BatchNormalization(),
#         Dropout(0.4),  # Increased dropout
        
#         # Bidirectional(LSTM(128, return_sequences=True)),
#         # Bidirectional(LSTM(64)),
        
#         LSTM(128, return_sequences=True),
#         LSTM(64),
        
#         Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Added L2 regularization
#         Dropout(0.5),
#         Dense(64, activation='relu', kernel_regularizer=l2(0.01)),  # Added L2 regularization
#         Dropout(0.4),  # Increased dropout
#         Dense(num_classes, activation='softmax')
#     ])

#     # Compile the model with Adam optimizer and learning rate scheduler
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#     audio_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     # Callbacks: Early stopping, learning rate scheduler, and model checkpoint
#     callbacks = [
#         tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.001, mode='min', restore_best_weights=True, verbose=1),
#         tf.keras.callbacks.ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)  # Learning rate scheduler
#     ]

#     # Train the model (optional class weights can be used if dataset is imbalanced)
#     # class_weight = {0: 1.0, 1: 2.0, 2: 1.5, ...}  # Adjust class weights based on class distribution
#     history = audio_model.fit(
#         X_train, y_train, 
#         epochs=100, 
#         batch_size=32, 
#         validation_data=(X_test, y_test), 
#         callbacks=callbacks
#         # ,class_weight=class_weight  # Uncomment if class weights are to be used
#     )

#     # Evaluate the model on the test set
#     loss, accuracy = audio_model.evaluate(X_test, y_test)
#     print(f"Audio Model Test Accuracy: {accuracy * 100:.2f}%")

#     return audio_model, history

# # Train the audio model with pre-extracted features
# audio_model, history = train_audio_model(x_train, y_train, x_test, y_test, num_classes=6)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense,GlobalMaxPooling1D, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2

def build_model(x_train, y_train, x_test, y_test, num_classes):
    
    input_shape = (100, x_train.shape[2])
    audio_model = Sequential([
        # Convolutional layer
        Conv1D(filters=64, kernel_size=3, activation='relu', 
               kernel_regularizer=l2(0.01), padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        Conv1D(filters=128, kernel_size=3, activation='relu', 
               kernel_regularizer=l2(0.01), padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        
        # BiLSTM layer
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.2),
        
        Bidirectional(LSTM(64)),
        Dropout(0.2),

        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    audio_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = audio_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, min_delta=0.01, mode='min', restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True)
    ])

    # Evaluate the model on the test set
    loss, accuracy = audio_model.evaluate(x_test, y_test)
    print(f"Audio Model Test Accuracy: {accuracy * 100:.2f}%")

    return audio_model, history

# Train the audio model with pre-extracted features
audio_model, history = build_model(x_train, y_train, x_test, y_test, num_classes=6)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Assuming `Y` is the original list of labels (before one-hot encoding)
# Fit the LabelEncoder on the original labels
label_encoder = LabelEncoder()
label_encoder.fit(np.argmax(y_train, axis=1))  # Fit LabelEncoder on the integer-encoded classes

# Since `y_test` is one-hot encoded, convert it back to the original class labels
y_true = np.argmax(y_test, axis=1)

# Predict the labels for the test set
y_pred = audio_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions from one-hot encoding to class labels

# Function to plot the confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# Get the unique class labels using the fitted label encoder
class_labels = label_encoder.classes_

# Plot the confusion matrix
plot_confusion_matrix(y_true, y_pred_classes, class_labels=class_labels)


In [14]:
flat_features = X.reshape(X.shape[0], -1)

In [93]:
plp_features = X
labels = Y

In [ ]:
# Group by the 'Emotions' column and pick the first sample for each emotion
samples = data_path.groupby('Emotions').first().reset_index()

# Display the resulting dataframe
print(samples)

In [ ]:
samples['Path'][0]

In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display

audio_file = samples['Path'][0]
y, sr = librosa.load(audio_file, sr=16000)
plt.figure(figsize=(12, 4))
librosa.display.waveshow(y, sr=sr)
plt.title('Waveform of the Audio Sample')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# plps = librosa.feature.plp(y=y, sr=sr, n_plp=13)
plps = extract_plp_features(data, sr=sr)
plt.figure(figsize=(12, 6))
librosa.display.specshow(plps, sr=sr, x_axis='time', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('plp Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('plp Coefficients')
plt.show()


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Load the audio file
audio_file = samples['Path'][0]  # Replace with your audio file path
y, sr = librosa.load(audio_file, sr=None)

# Remove silences at the start and end
y_trimmed, _ = librosa.effects.trim(y)

# Calculate time array for waveform
time = np.linspace(0, len(y_trimmed) / sr, len(y_trimmed))

# Create Mel spectrogram
S = librosa.feature.melspectrogram(y=y_trimmed, sr=sr, n_mels=128, fmax=8000)

# Create PLP spectrogram
PLPs = extract_plp_features(y_trimmed, sr)

# Create spectrogram using STFT
D = librosa.amplitude_to_db(np.abs(librosa.stft(y_trimmed)), ref=np.max)

# Create the figure with 4 subplots
fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)

# Plot the waveform
axs[0].set_title('Waveform')
axs[0].plot(time, y_trimmed, color='blue')
axs[0].set_ylabel('Amplitude')

# Plot the spectrogram
axs[1].set_title('Spectrogram (STFT)')
img1 = axs[1].imshow(
    D, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 0, sr // 2], 
    cmap='viridis'
)
axs[1].set_ylabel('Frequency (Hz)')
fig.colorbar(img1, ax=axs[1], format="%+2.0f dB")

# Plot the Mel spectrogram
axs[2].set_title('Mel Spectrogram')
mel_time = np.linspace(0, len(y_trimmed) / sr, S.shape[1])  # Align Mel spectrogram timeline
img2 = axs[2].imshow(
    librosa.power_to_db(S, ref=np.max), 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 0, 8000], 
    cmap='magma'
)
axs[2].set_ylabel('Frequency (Hz)')
fig.colorbar(img2, ax=axs[2], format="%+2.0f dB")

# Plot the PLP spectrogram
axs[3].set_title('PLP Spectrogram')
plp_time = np.linspace(0, len(y_trimmed) / sr, PLPs.shape[1])  # Align PLP spectrogram timeline
img3 = axs[3].imshow(
    PLPs, 
    aspect='auto', origin='lower', 
    extent=[0, len(y_trimmed) / sr, 1, PLPs.shape[0]], 
    cmap='cividis'
)
axs[3].set_ylabel('PLP Coefficients')
fig.colorbar(img3, ax=axs[3], format="%+2.0f dB")

# Add a shared X-axis label
plt.xlabel('Time (s)')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
plps.shape
labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming PLP features are in `plps` of shape (721, 13)
# Create a DataFrame with PLP features
plp_df = pd.DataFrame(plps, columns=[f'PLP {i+1}' for i in range(plps.shape[1])])

# Plot the boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=plp_df)
plt.title('Boxplot of PLP Coefficients')
plt.xlabel('PLP Coefficient Index')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# plp_features.shape
plp_features.shape, labels.shape

In [64]:
labels = labels.flatten()  # Ensures a 1D array


In [ ]:


# Compute mean across time steps for each sample
mean_plp = np.mean(plp_features, axis=1)  # Shape: (n_samples, n_coefficients)

# Convert to DataFrame for easier plotting
df = pd.DataFrame(mean_plp, columns=[f"PLP {i+1}" for i in range(mean_plp.shape[1])])
df['Emotion'] = labels

# Melt for grouped bar plot
df_melted = df.melt(id_vars='Emotion', var_name='PLP Coefficient', value_name='Mean Value')

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(data=df_melted, x='PLP Coefficient', y='Mean Value', hue='Emotion', ci="sd")
plt.title('Mean PLP Coefficients Across Emotions')
plt.xticks(rotation=45)
plt.legend(title='Emotion')
plt.show()

In [ ]:
# Select only the desired plp coefficients
selected_plp = ["plp 1", "plp 2", "plp 4", "plp 6"]
df_selected = df.melt(id_vars='Emotion', var_name='plp Coefficient', value_name='Mean Value')
df_selected = df_selected[df_selected['plp Coefficient'].isin(selected_plp)]

# Plot the selected coefficients
plt.figure(figsize=(12, 6))
sns.barplot(data=df_selected, x='plp Coefficient', y='Mean Value', hue='Emotion', ci="sd")
plt.title('Mean of Selected plp Coefficients Across Emotions')
plt.xticks(rotation=45)
plt.legend(title='Emotion')
plt.show()
